<a href="https://colab.research.google.com/github/Abhinavsingh0007/Twitter_Sentiment_Analysis/blob/main/TWITEER_SENTIMENT_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installation of Kaggle library
!pip install kaggle

Uploading kaggle .json file


In [ ]:
#configuring the path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing twitter sentiment dataset

In [ ]:
#API to fetch twitter sentiment dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 77% 62.0M/80.9M [00:00<00:00, 223MB/s]
100% 80.9M/80.9M [00:00<00:00, 225MB/s]


In [ ]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset= '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The data is extracted')

The data is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing


In [ ]:
twitter_dataset=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
#cheking the number of row and column in data
print(twitter_dataset.shape)

twitter_dataset.head()

(1599999, 6)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming  the column and reading the dataset again
column_name= ['target','id','date','flag','user','text']
twitter_dataset=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_name,encoding='ISO-8859-1')
#new dataset is created intially it was taking the 1st row as every column name
print(twitter_dataset.shape)
twitter_dataset.head()

(1600000, 6)


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#check for missing value
twitter_dataset.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
#checking the distribution of values in column
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
# replacing target value from 4 to 1
twitter_dataset.replace({'target':{4:1}},inplace=True)
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


0---> Negagtive Tweet



1---> Positive Tweet


In [ ]:
# Stemming process
port_stem= PorterStemmer()
def stemming(content):
  stemmed_count=re.sub('[^a-zA-z]',' ',content)
  stemmed_count=stemmed_count.lower()
  stemmed_count=stemmed_count.split()
  stemmed_count=[port_stem.stem(word) for word in stemmed_count  if not word in stopwords.words('english')]
  stemmed_count= ' '.join(stemmed_count)

  return stemmed_count

In [ ]:
twitter_dataset['stemmed_content']= twitter_dataset['text'].apply(stemming)


In [ ]:
twitter_dataset.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_dataset.stemmed_content)

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_dataset.target)

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# from tkinter import Y
#separating the data and label
X= twitter_dataset['stemmed_content'].values
Y= twitter_dataset['target'].values

In [ ]:
print(X)


['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
#spliting the data for training and testing
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)


In [ ]:
print(X_test.shape ,X_train.shape)

(320000,) (1280000,)


In [ ]:
#convert the textual value into numberical value
vectorizer=TfidfVectorizer()
# vectorizer.fit(X_train)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)


In [ ]:
print(X_train)

  (0, 464940)	0.4483581441053542
  (0, 458421)	0.27228112326228093
  (0, 372989)	0.3583819096048567
  (0, 247667)	0.4215060595161854
  (0, 194823)	0.5274064910069903
  (0, 116318)	0.3749338694119814
  (1, 169453)	1.0
  (2, 465311)	0.33480686186877023
  (2, 455221)	0.3296073093697297
  (2, 429906)	0.15168139293444055
  (2, 427967)	0.18706371586911208
  (2, 427034)	0.32103058232997417
  (2, 304223)	0.1678648733603879
  (2, 281186)	0.24137202118772233
  (2, 187474)	0.16194060015844067
  (2, 159240)	0.18800868496584808
  (2, 140239)	0.20289994039686138
  (2, 137276)	0.29069581876534506
  (2, 132152)	0.1891875792321668
  (2, 116318)	0.45905705320412793
  (2, 83506)	0.31303964332256906
  (3, 432396)	0.27084008410986327
  (3, 427034)	0.29025988953598614
  (3, 408311)	0.3941744062368763
  (3, 181609)	0.3745702863460755
  :	:
  (1279996, 392260)	0.3596073367651944
  (1279996, 335154)	0.21190629789318696
  (1279996, 306973)	0.1792858905560917
  (1279996, 250955)	0.5589110869092249
  (1279996, 25

In [ ]:
print(X_test)

  (0, 442233)	0.17903569516819148
  (0, 429906)	0.3140986667345934
  (0, 419078)	0.34884649404705365
  (0, 408024)	0.21977439030363208
  (0, 294440)	0.17813419553395948
  (0, 285841)	0.4532311769636366
  (0, 180544)	0.2803743470609849
  (0, 146252)	0.23682534421364126
  (0, 140294)	0.25509664355434203
  (0, 112919)	0.367081693610678
  (0, 72943)	0.2678484639307855
  (0, 34872)	0.16236726920078096
  (0, 17977)	0.17180827032540266
  (1, 385085)	0.24302725359248564
  (1, 366102)	0.4820488545932507
  (1, 270721)	0.28432607616832933
  (1, 229396)	0.40557814327518943
  (1, 153709)	0.5722666578553444
  (1, 17977)	0.2084958588978494
  (1, 8801)	0.30376315944222493
  (2, 420861)	0.4330300886388968
  (2, 270795)	0.2562344190631401
  (2, 192882)	0.5885804190746107
  (2, 95488)	0.363168936008175
  (2, 38175)	0.37941511785250515
  :	:
  (319994, 130897)	0.4539824479197656
  (319995, 466849)	0.3209500415545892
  (319995, 442233)	0.22620821621683035
  (319995, 437352)	0.23804373106127552
  (319995, 3

In [ ]:
#training the machine learning Model
#using Logistic regression
model=LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Accuracy score


In [ ]:
#accuracy score on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train , X_train_prediction)

In [ ]:
print("Accuracy score on training data = ", training_data_accuracy)


Accuracy score on training data =  0.8117125


In [ ]:
#accuracy score on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score( Y_test , X_test_prediction)


In [ ]:
print("Accuracy score on testing data = ", test_data_accuracy)


Accuracy score on testing data =  0.778553125


Model Accuracy = 77.8%


saving the trained model


In [ ]:
import pickle

In [ ]:
filename= 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))


Using the saved model for future prediction

In [ ]:
#loading the save model
loaded_model= pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new=X_test[2000]
print(Y_test[2000])
prediction=loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')